In [6]:
# Importamos las librerias que vamos as utilizar

import os
import pandas as pd

In [7]:
# Establecemos las rutas de los directorios con los que vamos a trabajar

cwd = os.getcwd() #
BASE_DIR = os.path.dirname(cwd) 
DATA_DIR = os.path.join(BASE_DIR, "files/MLOpsReviews") 
CACHE_DIR = os.path.join(BASE_DIR, "files/cache") 
RATING_DIR = os.path.join(BASE_DIR, "files/MLOpsReviews/ratings") 
os.makedirs(CACHE_DIR, exist_ok=True) 

In [8]:
# Creamos listas vacias con las que vamos a trabajar
my_dataframes = [] 
ratings = [] 

csv_files = [x for x in os.listdir(DATA_DIR) if x.endswith(".csv")] # Creamos una lista de los archivos dentro del directorio de la variable "DATA_DIR"
                                                                    # Se añade si cumple la condicion de que termine en ".csv"

# Vamos a iterar en cada archivo dentro de la carpeta, vamos a extraer la plataforma de origen, asignar una id (parte de la Propuesta de trabajo) 
# Y guardamos los dataframes en una lista
for filename in csv_files: 
    platform = filename.replace(".csv", "") 
    csv_path = os.path.join(DATA_DIR, filename) 
    this_df = pd.read_csv(csv_path, index_col=False) 
    this_df['id'] = platform[0]+this_df["show_id"]
    this_df['platform'] = platform 
    this_df.drop(columns='show_id', inplace=True) 
    my_dataframes.append(this_df) # 

rating_files = [x for x in os.listdir(RATING_DIR) if x.endswith(".csv")]  # Creamos una lista de los archivos dentro del directorio de la variable "RATING_DIR"
                                                                          # Se añade si cumple la condicion de que termine en ".csv"


# Volvemos a iterar, pero en otra carpeta que contiene los ratings
for filename in rating_files:
    csv_path = os.path.join(RATING_DIR, filename) 
    this_df = pd.read_csv(csv_path, index_col=False) 
    ratings.append(this_df) 

In [9]:
# Concatenamos las listas de dataframes 
all_dataframes = pd.concat(my_dataframes)
all_ratings = pd.concat(ratings)

# /////////////////////////

In [10]:
def removeNulls(dataframe):
    dataframe = dataframe["rating"].fillna('g', inplace=True)
    return dataframe
# Creamos una funcion para reemplazar valores NaN con la letra 'g' en la columna 'rating'

removeNulls(all_dataframes)

In [11]:
def standarize(dataframe):
    dataframe["date_added"] = pd.to_datetime(dataframe["date_added"])
    return dataframe
# Creamos uan funcion para establecer el formate de fecha como "YY-MM-DD"

standarize(all_dataframes)

,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,id,platform
0,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,2021-03-30,2014,g,113 min,"Comedy, Drama",A small fishing village must procure a local d...,as1,amazon
1,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,2021-03-30,2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...,as2,amazon
2,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,2021-03-30,2017,g,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...,as3,amazon
3,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,2021-03-30,2014,g,69 min,Documentary,"Pink breaks the mold once again, bringing her ...",as4,amazon
4,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,2021-03-30,1989,g,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...,as5,amazon
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8802,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,2019-11-20,2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a...",ns8803,netflix
8803,TV Show,Zombie Dumb,NaN,NaN,NaN,2019-07-01,2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g...",ns8804,netflix
8804,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,2019-11-01,2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...,ns8805,netflix
8805,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,2020-01-11,2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero...",ns8806,netflix


In [12]:
def lowercase(dataframe):
    dataframe = dataframe.apply(lambda x: x.astype(str).str.lower()
                         if (x.dtype == 'object') else x)
    return dataframe
# Funcion para poner en minuscula los strings

all_dataframes = lowercase(all_dataframes)

In [13]:
def durationSplit(dataframe):
    dataframe[['duration_int', 'duration_type']] = dataframe['duration'].str.split(expand=True)
    dataframe['duration_int'] = dataframe['duration_int'].astype(float).astype('Int64')
    dataframe['duration_type'] = dataframe['duration_type'].replace(['seasons', 'season'], 'season')
    dataframe.drop(columns='duration', inplace=True)
    return dataframe
# Funcion para separar la columna "duration" en dos columnas distintas, y quitamos el plural del valor "seasons"

all_dataframes = durationSplit(all_dataframes)

In [14]:
# Vamos a sacar el promedio de los ratings, renombramos algunas columnas y eliminamos otras columnas que no necesitamos
mean = all_ratings.groupby("movieId")["rating"].mean()
all_ratings["rating"] = all_ratings["rating"].groupby([all_ratings["movieId"]]).transform('mean')
all_ratings.drop_duplicates(subset=["movieId"], inplace=True)
all_ratings.drop(columns=['userId','timestamp'], inplace=True)
all_ratings.rename(columns={"rating": "score", "movieId": "id"}, inplace=True)
all_ratings

,score,id
0,3.549127,as680
1,3.556886,ns2186
2,3.604124,hs2381
3,3.547085,ns3663
4,3.532158,as9500
...,...,...
182025,3.470833,ns4069
195443,3.478632,ds619
198037,3.506757,ns7976
240502,3.551829,ns1837


In [15]:
# Establecemos la ruta y el nombre del archivo que vamos a guardar
dataset = os.path.join(CACHE_DIR, 'stream-data.csv')

# Concatenamos los dataframes y guardamos el csv
final_dataframe = pd.merge(all_dataframes, all_ratings, on='id',  how="inner")
final_dataframe.to_csv(dataset, index=False)

In [16]:
final_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22998 entries, 0 to 22997
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   type           22998 non-null  object        
 1   title          22998 non-null  object        
 2   director       22998 non-null  object        
 3   cast           22998 non-null  object        
 4   country        22998 non-null  object        
 5   date_added     13444 non-null  datetime64[ns]
 6   release_year   22998 non-null  int64         
 7   rating         22998 non-null  object        
 8   listed_in      22998 non-null  object        
 9   description    22998 non-null  object        
 10  id             22998 non-null  object        
 11  platform       22998 non-null  object        
 12  duration_int   22516 non-null  Int64         
 13  duration_type  22516 non-null  object        
 14  score          22998 non-null  float64       
dtypes: Int64(1), dateti